In [1]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from simplemma import text_lemmatizer
from nltk import sent_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

In [3]:
df = pd.read_csv(r"C:\Users\ozdem\Jupyter Notebook\dataset.csv")

In [4]:
df.head()

,Unnamed: 0,Link,Başlık,Body_text,Summarized_Text,TimeStamp,Kategori
0,0,https://www.ensonhaber.com/ekonomi/moodys-turk...,"\nMoody's, Türkiye için değerlendirme yapmadı\...",Uluslararası kredi derecelendirme kuruluşu Moo...,Haftalık Özet,NaN,ekonomi
1,1,https://www.ensonhaber.com/ekonomi/cevdet-yilm...,\nCevdet Yılmaz'dan asgari ücret ve emekliye z...,\n Bu videoyu izlemek için ...,Haftalık Özet,NaN,ekonomi
2,2,https://www.ensonhaber.com/ekonomi/mersinde-ha...,\nMersin'de hasat başladı! Bahçede 16 lira ola...,\n Bu videoyu izlemek için ...,Haftalık Özet,NaN,ekonomi
3,3,https://www.ensonhaber.com/ekonomi/turkiyenin-...,\nTürkiye'nin 2022'de AB ülkeleri içindeki sat...,"Türkiye İstatistik Kurumu (TÜİK), 2022 yılında...",Haftalık Özet,NaN,ekonomi
4,4,https://www.ensonhaber.com/ekonomi/cevdet-yilm...,\nCevdet Yılmaz: Yılı tek haneli işsizlik raka...,\n Bu videoyu izlemek için ...,Haftalık Özet,NaN,ekonomi


In [5]:
stopWords = set(stopwords.words('turkish'))


In [6]:
def clean_text(text):
    
    text = text.replace("Bu videoyu izlemek için lütfen JavaScript'i etkinleştirin", " ")
    text = text.replace("[/inlinetweet]", "")
    text = text.lower()
    text = re.sub("[0-9]+", "", text)
    text = re.sub("’|“|”|‘|–|—", " ", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = text_lemmatizer(text, lang="tr")
    text = [re.sub("[0-9]+", "", element) for element in text]
    text = " ".join([word for word in text if word not in stopWords])
    
    
    turkish_punctuation = ".,;:!?()[]{}'\"<>-_+=*&^%$#@~`|\\/"
    translate_table = str.maketrans("", "", turkish_punctuation)
    text = text.translate(translate_table)
    
    return text

In [7]:
df["clean"] = df["Body_text"].apply(lambda row: clean_text(row))

In [8]:
df["clean"][578] # Metinler temizlenmiş gibi duruyor.

' in güç  temas düzenle müsi ad vizyoner  zirve  ti m show center  düzenle  bura bir konuş gerçek borsa i stanbul genel müdür korkmaz ergun  borsa i stanbul  yatırım yap vatandaş sayı art bir kazanım ol belir  yatırım uzun vade yatırım perspektif kazan hedefle ilet  şirket yatırım ilgi cevap ver ad ürün hizmet çeşit sayı artır devam et aktar ergun  katılım finans geliş yönelik adım at devam et belir  şirket  borsa davet et  korkmaz ergun  etkin yap konuş  borsa i stanbul  şirket yatırım fark finansman yöntem yatırım araç sun söyle  piyasa ihtiyaç doğrultu ürün yelpaze çeşitlendirdik anlat ergun  açıkla bulun  hız güven bir şekil kesinti hizmet sun devam et  son yıl şirket yatırım göster yoğun ilgi iş bir şevk bağla  borsa i stanbul ol ülke büyü temel dinamo ol üretim  yatırım  ihracat yap  istihdam oluş şirket  büyü finansman halk arz yol sağla tek tek ziyaret et borsa davet et  i ş yap model değiş  ergun  borsa i stanbul ol iş yap model değiş dikkat çek  son üç yıl hacim hak bilgi ver

In [9]:
df["clean_label"] = df["Kategori"].apply(lambda row: clean_text(row)) #ic-haber seklindeki etiketi ichaber yaptık.

In [10]:
X = df.clean.to_numpy()
y = df.clean_label.to_numpy()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5)

In [12]:
print(len(X_train), len(X_test))

2307 577


In [13]:
X_train[0]

'abd eski başkan donald trump  ın destek  kongre bina işgal et başla gerilim sıra trump ın facebook  twitter diğer birçok sosyal medya hesap kapa  elon musk  ın twitter  ı satın al ardından abd eski başkan donald trump  a geri dön yol aç hesap yeniden aktif et  donald trump geri döndüabd eski başkan donald trump  eski ad twitter ol x sosyal medya platform geri dön  son ocak  paylaşım yap sosyal medya platform x  e geri dön trump  ın ilk paylaşım  seçim müdahale  asla teslim ol  yazılı sabıka fotoğraf bağış toplam sayfa yönlendir internet site link yer al  trump  georgia eyalet seçim sonuç müdahale bulun suçla hazırla iddianame kapsam teslim ol  dava kapsam bin dolar ol belirle kefalet bedel öde trump  yaklaşık dakika iç hapishane ayır  kendi sosyal medya platform um bulamadıtrump  x hesap askı alın ardından  truth social  isim ver sosyal medya uygulama kullanım sun iş pek iste git  öte yan trump  geç yıl yap bir video röportaj trump ifade kullan  truth social  ol  elon musk  ı sev  yap

In [14]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [15]:
model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

MultinomialNB()

In [16]:
sinif = ["ekonomi",
         "teknoloji",
         "otomobil",
         "ichaber"]

# Eğitim kümesindeki doğruluğu yazdır
print("NB eğitim doğruluğu:", model_NB.score(X_train, y_train))

# Test kümesindeki doğruluğu yazdır
print("NB test doğruluğu:", model_NB.score(X_test, y_test))

# Eğitim kümesinde tahminler yap
predictions_train = model_NB.predict(X_train)

# Her sınıf için ayrı F1 skorunu ekrana yazdır
f1_train = f1_score(y_train, predictions_train, average=None) #Birden fazla sınıf olduğu için f1 score'u for ile buluyoruz.
for i, f1 in enumerate(f1_train):
    print(f"NB Eğitim F1, Sınıf {sinif[i]} : {f1}")

# Test kümesinde tahminler yap
predictions_test = model_NB.predict(X_test)

# Her sınıf için ayrı F1 skorunu ekrana yazdır
f1_test = f1_score(y_test, predictions_test, average=None)
for i, f1 in enumerate(f1_test):
    print(f"NB Test F1, Sınıf {sinif[i]}: {f1}")


NB eğitim doğruluğu: 0.8777633289986996
NB test doğruluğu: 0.8838821490467937
NB Eğitim F1, Sınıf ekonomi : 0.8938193343898575
NB Eğitim F1, Sınıf teknoloji : 0.8527291452111225
NB Eğitim F1, Sınıf otomobil : 0.8616666666666668
NB Eğitim F1, Sınıf ichaber : 0.8975444538526673
NB Test F1, Sınıf ekonomi: 0.9283489096573209
NB Test F1, Sınıf teknoloji: 0.8549019607843138
NB Test F1, Sınıf otomobil: 0.851063829787234
NB Test F1, Sınıf ichaber: 0.8918918918918919


In [47]:
# şimdi dışarıdan bir teknoloji haberi aldım onu modelde deneyeceğiz.
metin = """
Yapay zeka son 1 yıldır hayatımıza tamamen entegre olmasının ardından kendisini her alanda geliştirmeye devam ediyor.

Son olarak yapay zekalı yüz tanıma sisteminin İngiltere havalimanlarında kullanılmaya başlanmasıyla birlikte pasaport kontrolüne gerek kalmayacağı kaydedildi.

İngiltere İçişleri Bakanlığı'nın planlarına göre yüz tanıma sistemiyle birlikte, temassız seyahat ve 50 millet için yapay zeka kullanan Dubai'ye benzer bir "akıllı sınır" yaratılması hedefleniyor.

Sınır Gücü Genel Müdürü Phil Douglas, "Yeni program şu anda yaptığımızdan çok daha temassız yüz tanıma kullanacak.

İnsanlar hakkında önceden çok daha fazla bilgi sahibi olacağız. Daha önce Birleşik Krallık'ta bulunup bulunmadıklarını bileceğiz" dedi.

Douglas, "Göçmenlik yasalarına uyumlarının ne olduğunu bileceğiz. Ve güvenlik sistemlerimizde onlarla ilgili herhangi bir kayıt olup olmadığını bileceğiz. Bu yüzden uçağa binemeyecek bazı insanlar olacak." diye konuştu.

Yeni teknolojinin denemelerinin bu yıl içinde yapılacağı bildirildi.

Buna ek olarak, gelecekte pasaportların kullanımdan kaldırılmasını sağlayabilecek sınır ötesi dijital kimlikler de test ediliyor.
"""



In [48]:
metin_temiz = clean_text(metin) #metini temizledik
metin_temiz_list =[metin_temiz] # liste yaptık ki vectorizer kodumuz hata vermesin

In [49]:
vectorized_metin =  vectorizer.transform(metin_temiz_list) # vektör haline getirdik ve modelimize tahmin ettireceğiz.

In [50]:
print("NB tahmini :",  model_NB.predict(vectorized_metin)) # gerçekten de doğru tahmin etti.

NB tahmini : ['teknoloji']
